In [1]:
import onnx
from onnx2pytorch import ConvertModel

onnx_model = onnx.load('Librispeech_Tensorflow_model.onnx')
pytorch_model = ConvertModel(onnx_model)

C:\Users\achantrill\AppData\Local\anaconda3\envs\MachineLearning\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
C:\Users\achantrill\AppData\Local\anaconda3\envs\MachineLearning\lib\site-packages\onnx2pytorch\convert\layer.py:30: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at C:\cb\pytorch_1000000000000\w

In [2]:
import pickle

with open('Librispeech_Tensorflow_X_train.pkl','rb') as f:
    X_train = pickle.load(f)
with open('Librispeech_Tensorflow_y_train.pkl','rb') as f:
    y_train = pickle.load(f)
with open('Librispeech_Tensorflow_X_val.pkl','rb') as f:
    X_val = pickle.load(f)
with open('Librispeech_Tensorflow_y_val.pkl','rb') as f:
    y_val = pickle.load(f)
with open('Librispeech_Tensorflow_X_test.pkl','rb') as f:
    X_test = pickle.load(f)
with open('Librispeech_Tensorflow_y_test.pkl','rb') as f:
    y_test = pickle.load(f)

In [14]:
y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [3]:
pytorch_model.eval()

ConvertModel(
  (Gemm_sequential/dense/MatMul_Gemm__6:0): Linear(in_features=193, out_features=193, bias=True)
  (Relu_sequential/dense/Relu:0): ReLU(inplace=True)
  (Gemm_sequential/dense_1/MatMul_Gemm__7:0): Linear(in_features=128, out_features=193, bias=True)
  (Relu_sequential/dense_1/Relu:0): ReLU(inplace=True)
  (Gemm_sequential/dense_2/MatMul_Gemm__8:0): Linear(in_features=128, out_features=128, bias=True)
  (Relu_sequential/dense_2/Relu:0): ReLU(inplace=True)
  (Gemm_sequential/dense_3/MatMul_Gemm__9:0): Linear(in_features=251, out_features=128, bias=True)
  (Softmax_dense_3): Softmax(dim=-1)
)

In [20]:
import numpy as np
import torch

predicted = pytorch_model(torch.tensor(np.array([X_val[0]]), dtype=torch.float32)).argmax()

In [21]:
predicted

tensor(166)

In [7]:
y_test.argmax(1)

array([166, 166, 166, ..., 187, 187, 187], dtype=int64)

In [37]:
from torchinfo import summary

summary(pytorch_model, input_size=(1, 193))

Layer (type:depth-idx)                   Output Shape              Param #
ConvertModel                             [1, 251]                  --
├─Linear: 1-1                            [1, 193]                  37,442
├─ReLU: 1-2                              [1, 193]                  --
├─Linear: 1-3                            [1, 128]                  24,832
├─ReLU: 1-4                              [1, 128]                  --
├─Linear: 1-5                            [1, 128]                  16,512
├─ReLU: 1-6                              [1, 128]                  --
├─Linear: 1-7                            [1, 251]                  32,379
├─Softmax: 1-8                           [1, 251]                  --
Total params: 111,165
Trainable params: 111,165
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.11
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 0.44
Estimated Total Size (MB): 0.45

In [15]:
y_val

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [16]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, parameters, labels, device):
        self.parameters = parameters
        self.labels = labels
        self.device = device

    def __len__(self):
        return len(self.parameters)

    def __getitem__(self, idx):
        param = torch.tensor(self.parameters[idx], dtype=torch.float32).to(self.device)
        label = torch.tensor(self.labels[idx].argmax(), dtype=torch.long).to(self.device)
        return param, label

test_loader = DataLoader(CustomDataset(X_val, y_val, 'cpu'), batch_size=1)
pytorch_model.eval()
criterion = nn.CrossEntropyLoss()
test_loss_accumulator = 0.0
test_correct_accumulator = 0
test_total_samples = 0

with torch.no_grad():
    for data, targets in test_loader:
        outputs = pytorch_model(data)
        loss = criterion(outputs, targets)
        test_loss_accumulator += loss.item() * data.size(0)
        _, predicted = torch.max(outputs, 1)
        print(f"Predicted: {predicted}")
        print(f"Targets: {targets}")
        print()
        test_correct_accumulator += (predicted==targets).sum().item()
        test_total_samples += data.size(0)
    test_loss = test_loss_accumulator / test_total_samples
    test_accuracy = test_correct_accumulator / test_total_samples
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {(test_accuracy * 100):.2f}%")

Predicted: tensor([166])
Targets: tensor([166])

Predicted: tensor([166])
Targets: tensor([166])

Predicted: tensor([166])
Targets: tensor([166])

Predicted: tensor([166])
Targets: tensor([166])

Predicted: tensor([166])
Targets: tensor([166])

Predicted: tensor([166])
Targets: tensor([166])

Predicted: tensor([166])
Targets: tensor([166])

Predicted: tensor([166])
Targets: tensor([166])

Predicted: tensor([166])
Targets: tensor([166])

Predicted: tensor([166])
Targets: tensor([166])

Predicted: tensor([166])
Targets: tensor([166])

Predicted: tensor([166])
Targets: tensor([166])

Predicted: tensor([27])
Targets: tensor([27])

Predicted: tensor([27])
Targets: tensor([27])

Predicted: tensor([27])
Targets: tensor([27])

Predicted: tensor([27])
Targets: tensor([27])

Predicted: tensor([27])
Targets: tensor([27])

Predicted: tensor([27])
Targets: tensor([27])

Predicted: tensor([27])
Targets: tensor([27])

Predicted: tensor([27])
Targets: tensor([27])

Predicted: tensor([225])
Targets: te

In [23]:
print(y_val[0])
test_loader = DataLoader(CustomDataset(X_val, y_val, 'cpu'), batch_size=1)
data, targets = next(iter(test_loader))
targets

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


tensor([166])